# 인코더 LSTM과 디코터 LSTM으로 스마트 번역기 만들기

In [1]:
# 1. 패키지 수입
import numpy as np
import pandas as pd
from time import time

from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼 파라미터 (정확도나 속도에 영향을 미치는 값)
MY_HIDDEN = 128
MY_EPOCH = 500

In [2]:
# 2. 번역 데이터 불러오기
raw = pd.read_csv('trans_dataset/translate.csv', header=None)
eng_kor = raw.values.tolist() # 데이터 프레임을 list로 변환
print(len(eng_kor))
print(eng_kor[:3])

110
[['cold', '감기'], ['come', '오다'], ['cook', '요리']]


In [3]:
# 3. 영문 알파벳과 한글 알파벳 리스트 만들기

# 영문
# {c:i for i, c in enumerate(e_alpha)}
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']

# 한글
# [c for c in '감기오다요리복사...']가 들어있는 csv 파일 불러오기
# Series로 받아서 바로 list로 변환
k_alpha = pd.read_csv('trans_dataset/korean.csv', header = None)[0].tolist() 
alpha   = e_alpha + k_alpha
alpha_total_size = len(alpha)

print('영문과 한글 알파벳:\n', alpha)
print('알파벳 개수(One-Hot Encoding할 Size):', alpha_total_size)
print('영문 알파벳 개수:', len(e_alpha))
print('한글 알파벳 개수:', len(k_alpha))

영문과 한글 알파벳:
 ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
알파벳 개수(One-Hot Encoding할 Size): 171
영문 알파벳 개수: 29
한글 알파벳 개수: 142


In [4]:
# 4. 문자당 num를 갖는 dict 만들기
char_to_num = {c:i for i, c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [5]:
eng_kor[0]
print(char_to_num['c'], char_to_num['o'], char_to_num['i'], char_to_num['d'])
print([char_to_num[c] for c in eng_kor[0][0]])
print(char_to_num['감'], char_to_num['기'])
print([char_to_num[c] for c in eng_kor[0][1]])

5 17 11 6
[5, 17, 14, 6]
32 46
[32, 46]


In [6]:
# 원핫인코딩 방법1
# 이 코드에서는 적용 힘듦. display(pd.get_dummies([5, 3, 6]))
# 원핫인코딩 방법2
to_categorical([5, 3, 6], num_classes=10) # 0~9 인덱스 원핫인코딩

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [7]:
# 원핫인코딩 방법 3
np.eye(10)[[5, 3, 6]]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [8]:
# 5. Encoder 입력(영문 알파벳 ID를 One-Hot Encoding),
#    Decoder 입력("S한글" ID를 One-Hot Encoding)
#    Decoder 출력("한글E" ID)
def encoding(eng_kor = eng_kor):
    enc_in  = []
    dec_in  = []
    dec_out = [] # [[32, 57, 1], [62, 78, 1]] ...
    for data in eng_kor:
        # Encoding 입력 데이터(영어 → 숫자 → One-Hot Encoding)
        eng = [char_to_num[c] for c in data[0]]
        eng_one = np.eye(alpha_total_size)[eng]                         # sol 1)
        # eng_one = to_categorical(eng, num_classes = alpha_total_size) # sol 2)
        enc_in.append(eng_one)
        
        # Decoding 입력 데이터("S한글" → 숫자 → One-Hot Encoding)
        kor = [char_to_num[c] for c in 'S' + data[1]]
        kor_one = np.eye(alpha_total_size)[kor]
        dec_in.append(kor_one)
        
        # Decoder 출력 데이터 "한글E → 숫자"
        kor = [char_to_num[c] for c in data[1] + 'E']
        dec_out.append(kor)
    return enc_in, dec_in, dec_out

In [9]:
# sample=[['wood', '나무'], ['wood', '단어']]
# encoding(sample)

In [10]:
# 6. 전체 번역 데이터 처리
X_enc, X_dec, Y_dec = encoding(eng_kor)

# numpy로 전환 
X_enc = np.array(X_enc)
X_dec = np.array(X_dec)
Y_dec = np.array(Y_dec).reshape(-1, 3, 1) # 차원추가

# 최종 입출력 데이터 shape 확인
X_enc.shape, X_dec.shape, Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3, 1))

In [11]:
# 7. 모델 구현
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4, alpha_total_size))

# 출력, state_c, state_h
_, state_h, state_c = LSTM(units=MY_HIDDEN, # 128
                           return_state=True)(ENC_IN)
# 인코더와 디코더 연결
link = [state_h, state_c]

# 디코더 구현
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN,
              return_sequences=True)(DEC_IN, initial_state=link)

# 최종 출력층
DEC_OUT = Dense(units=alpha_total_size,
               activation='softmax')(DEC_MID)

# 모델 생성 구현
model = Model(inputs=[ENC_IN, DEC_IN],
             outputs=DEC_OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 128),        153600      ['input_1[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

In [12]:
# 8. 학습과정 설정 & 학습하기
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='rmsprop', metrics=['acc'])
begin = time()
model.fit([X_enc, X_dec],
         Y_dec,
         epochs=MY_EPOCH,
         verbose=1)
end = time()
print('학습시간 :', end-begin)

Epoch 1/500
4/4 [==============================] - 11s 14ms/step - loss: 5.1214 - acc: 0.1758
Epoch 2/500
4/4 [==============================] - 0s 12ms/step - loss: 4.9838 - acc: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 18ms/step - loss: 4.2736 - acc: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 10ms/step - loss: 3.4743 - acc: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 12ms/step - loss: 3.4182 - acc: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 6ms/step - loss: 3.3764 - acc: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 9ms/step - loss: 3.3443 - acc: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 10ms/step - loss: 3.3192 - acc: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 11ms/step - loss: 3.2920 - acc: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 11ms/step - loss: 3.2666 - acc: 0.3333
Epoch 11/500
4/4 [==============================] - 0s 17ms/step - los

4/4 [==============================] - 0s 9ms/step - loss: 0.9899 - acc: 0.8848
Epoch 89/500
4/4 [==============================] - 0s 10ms/step - loss: 0.9978 - acc: 0.8606
Epoch 90/500
4/4 [==============================] - 0s 6ms/step - loss: 0.9360 - acc: 0.9212
Epoch 91/500
4/4 [==============================] - 0s 12ms/step - loss: 0.8883 - acc: 0.9182
Epoch 92/500
4/4 [==============================] - 0s 6ms/step - loss: 0.8605 - acc: 0.9303
Epoch 93/500
4/4 [==============================] - 0s 12ms/step - loss: 0.8366 - acc: 0.9273
Epoch 94/500
4/4 [==============================] - 0s 7ms/step - loss: 0.8351 - acc: 0.9303
Epoch 95/500
4/4 [==============================] - 0s 11ms/step - loss: 0.7923 - acc: 0.9394
Epoch 96/500
4/4 [==============================] - 0s 7ms/step - loss: 0.7550 - acc: 0.9455
Epoch 97/500
4/4 [==============================] - 0s 10ms/step - loss: 0.7265 - acc: 0.9515
Epoch 98/500
4/4 [==============================] - 0s 7ms/step - loss: 0.7069

4/4 [==============================] - 0s 11ms/step - loss: 0.0241 - acc: 1.0000
Epoch 175/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0223 - acc: 1.0000
Epoch 176/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0217 - acc: 1.0000
Epoch 177/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0215 - acc: 1.0000
Epoch 178/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0214 - acc: 1.0000
Epoch 179/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0175 - acc: 1.0000
Epoch 180/500
4/4 [==============================] - 0s 11ms/step - loss: 0.0167 - acc: 1.0000
Epoch 181/500
4/4 [==============================] - 0s 11ms/step - loss: 0.0167 - acc: 1.0000
Epoch 182/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0152 - acc: 1.0000
Epoch 183/500
4/4 [==============================] - 0s 12ms/step - loss: 0.0152 - acc: 1.0000
Epoch 184/500
4/4 [==============================] - 0s 6ms/step - 

4/4 [==============================] - 0s 10ms/step - loss: 1.8962e-04 - acc: 1.0000
Epoch 260/500
4/4 [==============================] - 0s 7ms/step - loss: 1.7958e-04 - acc: 1.0000
Epoch 261/500
4/4 [==============================] - 0s 10ms/step - loss: 1.7031e-04 - acc: 1.0000
Epoch 262/500
4/4 [==============================] - 0s 6ms/step - loss: 1.6436e-04 - acc: 1.0000
Epoch 263/500
4/4 [==============================] - 0s 10ms/step - loss: 1.5292e-04 - acc: 1.0000
Epoch 264/500
4/4 [==============================] - 0s 7ms/step - loss: 1.4337e-04 - acc: 1.0000
Epoch 265/500
4/4 [==============================] - 0s 10ms/step - loss: 1.3491e-04 - acc: 1.0000
Epoch 266/500
4/4 [==============================] - 0s 7ms/step - loss: 1.3156e-04 - acc: 1.0000
Epoch 267/500
4/4 [==============================] - 0s 10ms/step - loss: 1.4309e-04 - acc: 1.0000
Epoch 268/500
4/4 [==============================] - 0s 6ms/step - loss: 1.2105e-04 - acc: 1.0000
Epoch 269/500
4/4 [==========

4/4 [==============================] - 0s 9ms/step - loss: 4.5267e-06 - acc: 1.0000
Epoch 343/500
4/4 [==============================] - 0s 6ms/step - loss: 4.3988e-06 - acc: 1.0000
Epoch 344/500
4/4 [==============================] - 0s 11ms/step - loss: 4.1456e-06 - acc: 1.0000
Epoch 345/500
4/4 [==============================] - 0s 6ms/step - loss: 3.9736e-06 - acc: 1.0000
Epoch 346/500
4/4 [==============================] - 0s 10ms/step - loss: 3.9389e-06 - acc: 1.0000
Epoch 347/500
4/4 [==============================] - 0s 10ms/step - loss: 3.7970e-06 - acc: 1.0000
Epoch 348/500
4/4 [==============================] - 0s 10ms/step - loss: 3.6442e-06 - acc: 1.0000
Epoch 349/500
4/4 [==============================] - 0s 11ms/step - loss: 3.6178e-06 - acc: 1.0000
Epoch 350/500
4/4 [==============================] - 0s 10ms/step - loss: 3.4668e-06 - acc: 1.0000
Epoch 351/500
4/4 [==============================] - 0s 13ms/step - loss: 3.4271e-06 - acc: 1.0000
Epoch 352/500
4/4 [========

4/4 [==============================] - 0s 11ms/step - loss: 1.0498e-06 - acc: 1.0000
Epoch 426/500
4/4 [==============================] - 0s 10ms/step - loss: 1.0458e-06 - acc: 1.0000
Epoch 427/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0317e-06 - acc: 1.0000
Epoch 428/500
4/4 [==============================] - 0s 15ms/step - loss: 1.0151e-06 - acc: 1.0000
Epoch 429/500
4/4 [==============================] - 0s 11ms/step - loss: 1.0035e-06 - acc: 1.0000
Epoch 430/500
4/4 [==============================] - 0s 6ms/step - loss: 9.9666e-07 - acc: 1.0000
Epoch 431/500
4/4 [==============================] - 0s 11ms/step - loss: 9.8438e-07 - acc: 1.0000
Epoch 432/500
4/4 [==============================] - 0s 7ms/step - loss: 9.7499e-07 - acc: 1.0000
Epoch 433/500
4/4 [==============================] - 0s 9ms/step - loss: 9.7607e-07 - acc: 1.0000
Epoch 434/500
4/4 [==============================] - 0s 5ms/step - loss: 9.6018e-07 - acc: 1.0000
Epoch 435/500
4/4 [==========

In [16]:
# 9. 모델 사용
test = [['cold', 'PP'],
        ['down', 'PP'],
        ['desk', 'PP'],
        ['love', 'PP'],
        ['olve', 'PP'],
        ['lvoe', 'PP'],
        ['loev', 'PP'],
        ['eovl', 'PP']]
enc_in, dec_in, dec_out = encoding(test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
enc_in.shape, dec_in.shape

((8, 4, 171), (8, 3, 171))

In [18]:
pred = model.predict([enc_in, dec_in])
pred

1/1 [==============================] - 1s 713ms/step


array([[[3.5013538e-15, 1.3664335e-17, 8.2382992e-15, ...,
         7.1005146e-12, 1.8007578e-12, 1.6852919e-08],
        [7.2764382e-15, 1.2074805e-06, 3.7176179e-15, ...,
         2.7470271e-10, 3.9477475e-12, 1.1615028e-08],
        [3.3510831e-19, 1.0000000e+00, 5.8489155e-19, ...,
         1.6783822e-14, 2.8448551e-17, 2.3339013e-16]],

       [[2.9096238e-13, 7.1258384e-18, 3.7024840e-13, ...,
         1.5397120e-12, 1.8757013e-13, 3.1309821e-08],
        [2.6247353e-13, 3.1521230e-09, 1.3605005e-13, ...,
         5.5523675e-10, 7.0630627e-13, 9.1980085e-10],
        [1.5630858e-15, 9.9999905e-01, 1.0969944e-15, ...,
         9.6570052e-12, 5.6490572e-14, 3.9979142e-13]],

       [[6.3177558e-14, 5.3747182e-19, 1.4861184e-13, ...,
         1.6783419e-14, 2.4661504e-09, 4.7765025e-14],
        [1.4807997e-13, 1.5511551e-12, 2.1921774e-13, ...,
         2.4649675e-12, 3.4264699e-11, 1.5123758e-12],
        [9.9243286e-14, 1.0000000e+00, 1.1983978e-13, ...,
         6.0994876e-11, 2

In [19]:
pred.shape

(8, 3, 171)

In [25]:
# alpha[32], char_to_num['감']
# 출력형식 : cold -> 감기(32, xx, 1)
for i in range(len(pred)):
    eng = test[i][0]
    hat = pred[i].argmax(axis=1)
    kor = alpha[hat[0]] + alpha[hat[1]]
    print('{} -> {}({})'.format(eng, kor, hat))

cold -> 감기([32 46  1])
down -> 아래([111  67   1])
desk -> 책상([149  97   1])
love -> 사랑([96 66  1])
olve -> 사랑([96 66  1])
lvoe -> 사랑([96 66  1])
loev -> 사랑([96 66  1])
eovl -> 도리([62 72  1])
